#ДЗ pro

Входные параметры:

Готовая архитектура сети CustomNet для распознования CIFAR-10

Задание:

1) Обучить сверточную сеть на датасете CIFAR-10 с разными функциями активациями, добавить слои свертки и пулинга, добавить нейронов и написать отчет об изменениях точностей.

2) Получить accuracy близкий к 0.8 и выше

In [0]:
import torch
import random
import numpy as np
import os
import torchvision.datasets

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True



#Load data

In [0]:
CIFAR_train = torchvision.datasets.CIFAR10('./',download=True,train=True)
CIFAR_test = torchvision.datasets.CIFAR10('./',download=True,train=False)

X_train = torch.FloatTensor(CIFAR_train.data)
y_train = torch.LongTensor(CIFAR_train.targets)

X_test = torch.FloatTensor(CIFAR_test.data)
y_test = torch.LongTensor(CIFAR_test.targets)


X_train /= 255.
X_test /=255.

# меняем порядок размерности
X_train = X_train.permute(0,3,1,2)
X_test = X_test.permute(0,3,1,2)  

Files already downloaded and verified
Files already downloaded and verified


#Model1

In [0]:
import torch.nn as nn
class CustomNet(torch.nn.Module):
  def __init__(self):
    super(CustomNet,self).__init__()

    self.features = nn.Sequential(
            nn.Conv2d(3, channels, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(channels),
        
            nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(channels),            
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(p = drop),
        
            nn.Conv2d(channels, channels*2, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(channels*2),
        
            nn.Conv2d(channels*2, channels*2, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(channels*2),            

            nn.Conv2d(channels*2, channels*2, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(channels*2),            
        
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
          
    self.classifier = nn.Sequential(
            nn.Dropout(p = drop),
            nn.Linear(channels*2 * 8 * 8, fc_units),
            nn.BatchNorm1d(fc_units),
            nn.ReLU(inplace=True),
            nn.Dropout(p = drop),

            nn.Linear(fc_units, fc_units),
            nn.BatchNorm1d(fc_units),
            nn.ReLU(inplace=True),
            nn.Dropout(p = drop),
            nn.Linear(fc_units, 10),
        
#             nn.Linear(fc_units, 10), #77

#             nn.Linear(fc_units, fc_units//2),
#             nn.BatchNorm1d(fc_units//2),
#             nn.ReLU(inplace=True),
#             nn.Dropout(p = drop),
#             nn.Linear(fc_units//2, 10),
        
        
        )
    self.activation = nn.Softmax()
  def forward(self,x):    
    x = self.features(x)
    x = x.view(x.size(0), -1)
    x = self.classifier(x)
    #x = self.activation(x)

    return x       

#Train model1

In [0]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

batch_size = 256
channels = 16
fc_units = 768
drop = .1

lenet5 = CustomNet()
lenet5 = lenet5.to(device)

loss = torch.nn.CrossEntropyLoss()
lr=1.0e-2
optimizer = torch.optim.Adam(lenet5.parameters(),lr=lr)



# tensors into gpu
X_test = X_test.to(device)
y_test = y_test.to(device)


num_epochs = 150    
for epoch in range(num_epochs):
  order = np.random.permutation(len(X_train))
  
  if epoch != 0 and epoch%3==0:    
    lr *= .8
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
        
    print ('lr set to {:.5f}:'.format(lr))
  
  train_preds = torch.tensor([],dtype=torch.uint8).to(device)
  train_accuracy = 0
  train_loss = 0
  for start_index in range(0,len(X_train), batch_size):        
    optimizer.zero_grad()
    batch_indexes = order[start_index:start_index+batch_size]
    
    X_batch = X_train[batch_indexes].to(device)
    y_batch = y_train[batch_indexes].to(device)
    
    preds = lenet5.forward(X_batch)        
    
    train_preds = torch.cat((train_preds,preds.argmax(dim=1) == y_batch), dim=0)
    
    loss_value = loss(preds, y_batch)
    
    train_loss += loss_value.item()
    
    loss_value.backward()
    
    optimizer.step()
  
  train_accuracy = train_preds.float().mean()
  
  test_preds = lenet5.forward(X_test)
  
  accuracy = (test_preds.argmax(dim=1) == y_test).float().mean().data.cpu()
  
  print('Epoch [{}/{}], Loss:{:.5f} , Accuracy:{:.5f}, Val Accuracy: {:.5f}'.format(epoch + 1, num_epochs, train_loss/(start_index+1),  train_accuracy, accuracy)  )
  

Epoch [1/150], Loss:0.00541 , Accuracy:0.49816, Val Accuracy: 0.61580
Epoch [2/150], Loss:0.00371 , Accuracy:0.66460, Val Accuracy: 0.69650
Epoch [3/150], Loss:0.00294 , Accuracy:0.73628, Val Accuracy: 0.73100
lr set to 0.00800:
Epoch [4/150], Loss:0.00234 , Accuracy:0.78932, Val Accuracy: 0.75810
Epoch [5/150], Loss:0.00193 , Accuracy:0.82568, Val Accuracy: 0.76260
Epoch [6/150], Loss:0.00161 , Accuracy:0.85256, Val Accuracy: 0.76540
lr set to 0.00640:
Epoch [7/150], Loss:0.00116 , Accuracy:0.89386, Val Accuracy: 0.77700
Epoch [8/150], Loss:0.00095 , Accuracy:0.91442, Val Accuracy: 0.77510
Epoch [9/150], Loss:0.00078 , Accuracy:0.92964, Val Accuracy: 0.76500
lr set to 0.00512:
Epoch [10/150], Loss:0.00052 , Accuracy:0.95400, Val Accuracy: 0.76970
Epoch [11/150], Loss:0.00042 , Accuracy:0.96092, Val Accuracy: 0.78080
Epoch [12/150], Loss:0.00040 , Accuracy:0.96346, Val Accuracy: 0.77620
lr set to 0.00410:
Epoch [13/150], Loss:0.00031 , Accuracy:0.97280, Val Accuracy: 0.77430
Epoch [14/

#Вывод

[Colab](https://colab.research.google.com/drive/1e14Vh74QYZpwYFqF3IVJEBlg2B8xklyp)

Использовал модель типа ConvNet с 5 слоями свертки (меньше сеть сделать не получилось, с увеличением числа каналов не хватает памяти GPU), и двух полносвязных слоев,  сеть достигает 80% к 76 эпохе.